In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

In [2]:
data= pd.read_csv(r"C:\Users\hsynt\Downloads\Selcuk Abi\Selcuk Abi\VeriSeti2.csv")

In [3]:
# Özellik ve Hedef Ayrımı
X = data.drop(columns=['Target_10', 'adjclose'])
y = data['Target_10']

# Zaman Serisi Formatına Getirme
def create_sequences(data, target, seq_length):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length].flatten())  # MLP için düzleştirilmiş giriş
        targets.append(target[i + seq_length])
    return np.array(sequences), np.array(targets)

SEQ_LENGTH = 10
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_seq, y_seq = create_sequences(X_scaled, y.values, SEQ_LENGTH)

# Eğitim ve test kümelerinin oluşturulması
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42, stratify=y_seq)

# PCA ve LDA Veri Hazırlığı
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

lda = LDA(n_components=1)
X_train_lda = lda.fit_transform(X_train, y_train)
X_test_lda = lda.transform(X_test)

# MLP Modeli Tanımlama
def build_mlp_model(input_shape):
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Performans Hesaplama
results = []
threshold = 60  # Underfitting için eşik

for reduction, X_tr, X_te, y_tr, y_te in zip(
    ['No PCA/LDA', 'PCA', 'LDA'],
    [X_train, X_train_pca, X_train_lda],
    [X_test, X_test_pca, X_test_lda],
    [y_train, y_train, y_train],
    [y_test, y_test, y_test]
):
    # MLP modelini oluştur ve eğit
    mlp_model = build_mlp_model(X_tr.shape[1])
    history = mlp_model.fit(X_tr, y_tr, epochs=10, batch_size=32, verbose=1, validation_data=(X_te, y_te))

    # Eğitim ve Test Kayıpları
    train_acc = history.history['accuracy'][-1] * 100
    test_acc = history.history['val_accuracy'][-1] * 100
    train_loss = history.history['loss'][-1]
    test_loss = history.history['val_loss'][-1]

    # Aşırı ve Yetersiz Öğrenme Kontrolü
    overfit = train_acc - test_acc > 5  # %5'ten fazla fark varsa aşırı öğrenme
    underfit = train_acc < threshold and test_acc < threshold

    results.append({
        'Reduction': reduction,
        'Model': 'MLP',
        'Train Accuracy (%)': train_acc,
        'Test Accuracy (%)': test_acc,
        'Train Loss': train_loss,
        'Test Loss': test_loss,
        'Overfitting': overfit,
        'Underfitting': underfit
    })

# Performans Sonuçlarını Görselleştirme
results_df = pd.DataFrame(results)

# Performans Sonuçları Tablosu
print("Performans Sonuçları Tablosu:")
print(results_df)

# Sonuçları Kaydetme
results_df.to_csv("mlp_overfitting_underfitting_results.csv", index=False)
print("\nSonuçlar 'mlp_overfitting_underfitting_results.csv' dosyasına kaydedildi.")

Epoch 1/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8228 - loss: 0.4126 - val_accuracy: 0.8647 - val_loss: 0.3149
Epoch 2/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8696 - loss: 0.3137 - val_accuracy: 0.8716 - val_loss: 0.3004
Epoch 3/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8730 - loss: 0.3014 - val_accuracy: 0.8738 - val_loss: 0.2983
Epoch 4/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8795 - loss: 0.2930 - val_accuracy: 0.8754 - val_loss: 0.2924
Epoch 5/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8741 - loss: 0.2978 - val_accuracy: 0.8749 - val_loss: 0.2919
Epoch 6/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8746 - loss: 0.2937 - val_accuracy: 0.8736 - val_loss: 0.2911
Epoch 7/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8767 - loss: 0.2879 - val_accuracy: 0.8743 - val_loss: 0.2938
Epoch 8/10
1249/1249 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8797 - loss: 0.2804 - 